In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load and read dataset

In [ ]:
import pandas as pd
import numpy as np
train_df = pd.read_csv("../input/commonlitreadabilityprize/train.csv",index_col="id")
test_df = pd.read_csv("../input/commonlitreadabilityprize/test.csv",index_col="id")
df = pd.concat([train_df, test_df])
train_df.shape, test_df.shape, df.shape

In [ ]:
test_rows = test_df.shape[0]
dfs = [train_df,test_df]
for df_ in dfs:
    temp_df = {"col":[],"count_unique":[],"unique":[],"null":[]}
    for col in df_.columns:
        temp_df["col"].append(col)
        temp_df["count_unique"].append(df_[col].nunique())
        temp_df["unique"].append(df_[col].unique())
        temp_df["null"].append(df_[col].isnull().sum())
    display(pd.DataFrame(temp_df))

In [ ]:
# look into excerpt data with null and notnul in license
from termcolor import colored
temp_df = df[df.license.notnull()].head(1)
for col in temp_df.columns:
    print(f"{colored(col,'red')}: {temp_df[col].values[0]}")

print()

temp_df = df[df.license.isnull()].head(1)
for col in temp_df.columns:
    print(f"{colored(col,'red')}: {temp_df[col].values[0]}")

# Transform and create new Features

In [ ]:
from nltk.tokenize import sent_tokenize
df["noof_sentences"] = df.excerpt.apply(lambda x: len(sent_tokenize(x)))

from nltk.tokenize import word_tokenize
df["noof_words"] = df.excerpt.apply(lambda x: len(word_tokenize(x)))

from nltk.corpus import stopwords
def get_stopwords(text):
    words = word_tokenize(text)
    return len([word for word in words if word in set(stopwords.words("english"))])
df["noof_stopwords"] = df.excerpt.apply(get_stopwords)

df["noof_chars"] = df.excerpt.apply(lambda x: len(x))
df["noof_space"] = df.excerpt.apply(lambda x: x.count(" "))
df["islegal"] = df.url_legal.apply(lambda x: False if pd.isnull(x) else True).astype("int")
df["islicence"] = df.license.apply(lambda x: False if pd.isnull(x) else True).astype("int")

**Incorroect words check count**

In [ ]:
'''
from nltk.corpus import wordnet
def get_wrongwords(text):
    wrongwords = []
    for i in "? , . ! ( ) - ` ;".split():
        text = text.replace(i, " ")
    for word in word_tokenize(text.rstrip()):
        if not wordnet.synsets(word):
            if word not in set(stopwords.words("english")):
                if not word[0].isupper():
                    if not word.isdigit():
                        wrongwords.append(word)
    return len(wrongwords)
df["noof_wrongwords"] = df.excerpt.apply(get_wrongwords)
df.noof_wrongwords.mean()
'''

**Incorrext grammer check count**

In [ ]:
from itertools import chain
#list(chain.from_iterable(df.noof_wrongwords.values))

# Data visualisation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numeric_columns = df.select_dtypes(exclude="object").columns
data = {"col":[],"corr":[]}
for col in numeric_columns:
    data["col"].append(col)
    data["corr"].append(abs(df[:-test_rows].target.corr(df[:-test_rows][col])))
pd.Series(data["corr"],index=data["col"]).sort_values().plot(kind="barh",title="Correlation")

In [ ]:
plt.figure(figsize=(10,30))
for i, col in enumerate(numeric_columns):
    plt.subplot(len(numeric_columns),2,i*2+1)
    sns.boxplot(x=df[:-test_rows][col])
    plt.subplot(len(numeric_columns),2,i*2+2)
    sns.lineplot(x=df[:-test_rows][col],y=df[:-test_rows].target)
sns.despine()

# Replace outlier

> Replace outlier tested not eorks and Drop outlier do improve model little

In [ ]:
def outlair(x):
    if x > upper_lim:
        return None
    elif x < lower_lim:
        return None
    else:
        return x
plt.figure(figsize=(10, 30))  
for i, col in enumerate(numeric_columns):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    upper_lim = q3 + 1.5 * iqr
    lower_lim = q1 - 1.5 * iqr
    indexes = df[:-test_rows].index
    df.loc[indexes,col] = df[:-test_rows][col].apply(outlair)
    
    # drop outlier
    df2 = df[:-test_rows].dropna(subset=[col],axis=0)
    df = pd.concat([df2,df[-test_rows:]])
    # visualise changes
    plt.subplot(len(numeric_columns),2,i*2+1)
    sns.boxplot(x=df[col]) 
    plt.subplot(len(numeric_columns),2,i*2+2)
    sns.lineplot(x=df[col],y=df.target)
print(len(train_df)-len(df[:-test_rows]),"rows dropped!")
sns.despine()

In [ ]:
df.isnull().sum(), df.shape, df2.shape

# Features scaling     X = (Xi - Xmean)/std

In [ ]:
# for col in list(set(numeric_columns) - set("islegal islicence".split())):
#     mean_, std_ = df[col].mean(), df[col].std()
#     df[col] = df[col].apply(lambda x: (x-mean_)/std_)

# Evaluate models (Predict standard_error)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from termcolor import colored
import warnings
warnings.filterwarnings("ignore",category=Warning)

def evaluate_models(X,y):
    models = [
        ("LIR",LinearRegression()),
#         ("LOR",LogisticRegression()),
        ("RFR",RandomForestRegressor(n_estimators=100)),
        ("XGB",XGBRegressor(learning_rate=0.01,max_depth=4,eval_metric="error",n_estimators=100,use_label_encoder=False)),
        ("SVR",SVR(gamma=0.1)),
        ("GBR",GradientBoostingRegressor(n_estimators=100)),
        ("DTR",DecisionTreeRegressor(max_leaf_nodes=500)),
#         ("LDA",LinearDiscriminantAnalysis()),
        ("KNR",KNeighborsRegressor(leaf_size=13,n_neighbors=29,p=1)),
#         ("GNB",GaussianNB(var_smoothing=0.0533669923120631)),
        ("ETR",ExtraTreesRegressor(max_depth=4,n_estimators=100)),
        ("BGR",BaggingRegressor(max_samples=0.2,n_estimators=100)),
    ]
    for name,model in models:
        results,times =[],5
        for i in range(times):
            train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=0.2,train_size=0.8,random_state=666)
            model.fit(train_X,train_y)
            pred = model.predict(test_X) #.round(decimals=0)
            results.append(np.sqrt(mean_squared_error(pred,test_y)))
        print(colored(f"{name} Model accuracy:","blue"),sum(results)/times)
        #print(colored("Results:","red"),results)

In [ ]:
X.head(1)

In [ ]:
numeric_columns = df.select_dtypes(exclude="object").columns.drop("target standard_error".split())
#numeric_columns= "noof_sentences noof_chars noof_wrongwords".split()
X = df[:-test_rows][numeric_columns]
y = df[:-test_rows]["standard_error"]

evaluate_models(X,y)

# Fill standard_error into test dataset

In [ ]:
#model = LinearRegression()
model = RandomForestRegressor(n_estimators=100,random_state=0)
model.fit(X,y)
pred = model.predict(df[-test_rows:][numeric_columns])
indexes = df[-test_rows:].index
df.loc[indexes,"standard_error"] = pred

# Evaluate models (Predict target)

In [ ]:
numeric_columns = df.select_dtypes(exclude="object").columns.drop("target".split())
X = df[:-test_rows][numeric_columns]
y = df[:-test_rows]["target"]
evaluate_models(X,y)

In [ ]:
#model = LinearRegression()
model = RandomForestRegressor(n_estimators=100, random_state=0)
model = BaggingRegressor(max_samples=0.2,n_estimators=100)
model.fit(X,y)
pred = model.predict(df[-test_rows:][numeric_columns])
output = pd.DataFrame({"id":df[-test_rows:].index,"target":pred})
output.to_csv("submission.csv", index=False)
pd.read_csv("./submission.csv").transpose()